# Importation des librairies

In [3]:
! pip install -q pandas-Profiling

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.4/324.4 kB 3.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.2/65.2 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.7/102.7 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 679.5/679.5 kB 2.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.3/293.3 kB 617.7 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.5/296.5 kB 592.9 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 1.0 MB/s eta 0:00:0000:0100:010m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 357.5/357.5 kB 844.9 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 1.3 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 1.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 1.3 MB/s eta 0:00:0000:0

In [16]:
import requests
import pandas as pd
from io import StringIO
from sklearn.impute import KNNImputer

# Importation des bases

## Import de la base de données de consommation d'électricité  et des données météorologiques via API de l'OpenDataSoft

L'éxécution de la requête API prend assez de temps

In [17]:
def get_data_ODS(dataset_name):
    """
    Fonction qui permet de récupérer des données via l'API d'OpenDataSoft
    Elle retourne un dataframe
    
    """
    df = pd.DataFrame()
    # URL de base pour accéder à l'API d'ODS
    base_url_ODS = "https://odre.opendatasoft.com/api/explore/v2.1"
    
    dataset_path = f"/catalog/datasets/{dataset_name}/exports/json?lang=fr&timezone=Europe%2FBerlin"
    url = f"{base_url_ODS}{dataset_path}"
    
    # Exécution de la requête GET
    
    response = requests.get(url)
    # Vérification si la requête a réussi
    if response.status_code == 200:
        # Extraction des données
        data = response.json()
        # Conversion des résultats en DataFrame
        df = pd.DataFrame(data)
    else:
        print(f"Erreur lors de la requête: {response.status_code}")

    return df


def get_data_ODSW(dataset_name):
    """
    Fonction qui permet de récupérer des données via l'API d'OpenDataSoft
    Elle retourne un dataframe
    
    """
    df = pd.DataFrame()
    # URL de base pour accéder à l'API d'ODS
    base_url_ODS = "https://public.opendatasoft.com/api/explore/v2.1"
    
    dataset_path = f"/catalog/datasets/{dataset_name}/exports/json?lang=fr&timezone=Europe%2FBerlin"
    url = f"{base_url_ODS}{dataset_path}"
    
    # Exécution de la requête GET
    
    response = requests.get(url)
    # Vérification si la requête a réussi
    if response.status_code == 200:
        # Extraction des données
        data = response.json()
        # Conversion des résultats en DataFrame
        df = pd.DataFrame(data)
    else:
        print(f"Erreur lors de la requête: {response.status_code}")

    return df

In [18]:
consumption_dataset_name = "consommation-quotidienne-brute-regionale"
weather_dataset_name = "donnees-synop-essentielles-omm"
data_consumption = get_data_ODS(consumption_dataset_name)
data_weather = get_data_ODSW(weather_dataset_name)

Erreur lors de la requête: 404


In [23]:
data_weather.head()

,numer_sta,date,pmer,tend,cod_tend,dd,ff,t,td,u,...,altitude,libgeo,codegeo,nom_epci,code_epci,nom_dept,code_dep,nom_reg,code_reg,mois_de_l_annee
0,07558,2010-01-05T10:00:00+01:00,100280.0,-50.0,5,260.0,1.5,275.75,275.75,100.0,...,712,Millau,12145,CC de Millau Grands Causses,241200567,Aveyron,12,Occitanie,76,1
1,61976,2010-01-05T10:00:00+01:00,100990.0,NaN,None,NaN,NaN,305.45,299.05,69.0,...,7,None,None,None,None,None,None,None,None,1
2,07027,2010-01-05T13:00:00+01:00,100720.0,-190.0,8,200.0,3.6,273.65,271.75,87.0,...,67,Carpiquet,14137,CU Caen la Mer,200065597,Calvados,14,Normandie,28,1
3,07110,2010-01-05T13:00:00+01:00,100750.0,-230.0,8,210.0,4.1,276.95,272.55,73.0,...,94,Guipavas,29075,Brest Métropole,242900314,Finistère,29,Bretagne,53,1
4,07591,2010-01-05T13:00:00+01:00,NaN,NaN,None,NaN,NaN,274.45,269.05,67.0,...,871,Embrun,05046,CC Serre-Ponçon,200067742,Hautes-Alpes,05,Provence-Alpes-Côte d'Azur,93,1


## Import d'une base de données complémentaire (celle des régions, communes, et départements de France

Cette importation va se faire à travers une requête via l'API de Datagouv

In [24]:
#Identifiant du dataset
dataset_id = "4a7c837bb6da8e363604082bcc8b2e504cf08038"
dataset_sha1 = "dbe8a621-a9c4-4bc3-9cae-be1699c5ff25"

# URL de base pour accéder à l'API
base_url = "https://www.data.gouv.fr/api/1/"

# Chemin pour accéder aux enregistrements du dataset
dataset_path = f"datasets/r/{dataset_sha1}?dataset={dataset_id}"

# Construction de l'URL complète
url = f"{base_url}{dataset_path}"

# Exécution de la requête GET
response = requests.get(url)

# Vérification si la requête a réussi
if response.status_code == 200:
    # Lecture du contenu CSV
    data_communes = pd.read_csv(StringIO(response.content.decode('utf-8')))
else:
    print(f"Erreur lors de la requête: {response.status_code}")

On enregistre la base pour ne plus avoir à éxécuter la requête API à chaque étape

In [25]:
data_consumption.to_csv('data_consumption.csv', index=False)
data_communes.to_csv('data_communes.csv', index=False)
data_weather.to_csv('data_weather.csv', index = False)

In [35]:
data_consumption = pd.read_csv('data_consumption.csv')
data_communes = pd.read_csv('data_communes.csv')
data_weather = pd.read_csv('data_weather.csv')

Les dataframes de consommation et météorologiques  étant sous forme de séries temporelles, on place les dates en index.

In [36]:
data_consumption['date'] = pd.to_datetime(data_consumption['date'])
data_consumption.set_index('date', inplace=True)

In [37]:
data_communes.isna().sum()

code_commune_INSEE          0
nom_commune_postal          0
code_postal                 0
libelle_acheminement        0
ligne_5                 35944
latitude                  269
longitude                 269
code_commune                7
article                 36621
nom_commune                 0
nom_commune_complet         0
code_departement            7
nom_departement           267
code_region               267
nom_region                267
dtype: int64

In [38]:
col_del = pd.Index(['date_heure', 'consommation_brute_gaz_grtgaz', 'statut_grtgaz', 'consommation_brute_gaz_terega', 'statut_terega','consommation_brute_gaz_totale','statut_rte', 'consommation_brute_totale'])
data_consumption.drop(col_del, axis=1,inplace=True)
data_consumption.isna().sum()

heure                                 0
code_insee_region                     0
region                                0
consommation_brute_electricite_rte    0
dtype: int64

NameError: name 'df' is not defined

In [58]:
df.head()

,heure,code_insee_region,region,consommation_brute_electricite_rte
date,,,,
2013-01-01,01:30,44,Grand Est,4840
2013-01-01,11:30,28,Normandie,3183
2013-01-01,11:00,27,Bourgogne-Franche-Comté,2280
2013-01-01,10:30,76,Occitanie,4392
2013-01-01,10:30,75,Nouvelle-Aquitaine,4636


In [39]:
data_consumption1 = data_consumption.groupby(['code_insee_region','date'], as_index = False)
data_consumption2 = data_consumption.groupby(['code_insee_region','date','heure'], as_index = False)

In [40]:
data_consumption1 = data_consumption1['consommation_brute_electricite_rte'].sum()
data_consumption2 = data_consumption2['consommation_brute_electricite_rte'].sum()

/tmp/ipykernel_498/3628389353.py:1: FutureWarning: A grouping was used that is not in the columns of the DataFrame and so was excluded from the result. This grouping will be included in a future version of pandas. Add the grouping as a column of the DataFrame to silence this warning.
  data_consumption1 = data_consumption1['consommation_brute_electricite_rte'].sum()
/tmp/ipykernel_498/3628389353.py:2: FutureWarning: A grouping was used that is not in the columns of the DataFrame and so was excluded from the result. This grouping will be included in a future version of pandas. Add the grouping as a column of the DataFrame to silence this warning.
  data_consumption2 = data_consumption2['consommation_brute_electricite_rte'].sum()


In [44]:
data_consumption1.head()

,code_insee_region,consommation_brute_electricite_rte
0,11,399392
1,11,492157
2,11,487111
3,11,470053
4,11,433732


## Aggrégation des consommations par date et par région

Imputation des variables manquantes : KNN Imputer

On se dit que les zones ayant des données semblables météorologiques "se ressemblent"

On vérifie d'abord que toutes les variables sont numériques; 

In [34]:
# Cas des données de consommation
data_consumption.dtypes

# Cas des données des communes
data_communes.dtypes


code_commune_INSEE       object
nom_commune_postal       object
code_postal               int64
libelle_acheminement     object
ligne_5                  object
latitude                float64
longitude               float64
code_commune            float64
article                  object
nom_commune              object
nom_commune_complet      object
code_departement         object
nom_departement          object
code_region             float64
nom_region               object
dtype: object

On convertit les colonnes

In [ ]:
cols = []
for col in cols:
    df[col] = df[col].astype(float)


On applique ensuite le KNN imputer

In [32]:

# Initialiser l'imputeur KNN
cons_imputer = KNNImputer(n_neighbors=2)
com_imputer = KNNImputer(n_neighbors=2)

# Appliquer l'imputation
data_consumption_imputed = pd.DataFrame(cons_imputer.fit_transform(data_consumption), columns=data_consumption.columns)
data_communes_imputed = pd.DataFrame(com_imputer.fit_transform(data_communes), columns = data_communes.columns)


ValueError: could not convert string to float: '2019-02-01T06:00:00+01:00'